In [1]:
# -*- coding: utf-8 -*-

import os
import httplib2
import sys
import google.oauth2.credentials
import google_auth_oauthlib.flow

from googleapiclient.discovery import build, build_from_document
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from apiclient.discovery import build
from oauth2client.tools import argparser, run_flow
from oauth2client.client import flow_from_clientsecrets
from oauth2client.file import Storage


import pandas as pd
import downloadPermission

import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import re

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pickle

from nltk.tokenize import word_tokenize # or use some other tokenizer
from nltk.corpus import stopwords
import string

In [2]:
dbname = 'youtubeSpon'
username = 'april' # change this to your username

engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
con = None
con = psycopg2.connect(database = dbname, user = username)
# query:
sql_query = """
SELECT * FROM "youtubeEmmy" WHERE sponsored <> 'None';
"""
youtube_data_from_sql = pd.read_sql_query(sql_query,con)
youtube_data_from_sql.tail()

,index,channel,title,video_id,description,captions,sponsored,fullCaptions
253,810,emmymadeinjapan,WINNERS Curry Set Giveaway,CQTp_Hh8qH4,Announcing the winners of the Popin Cookin' Ka...,"1\n00:00:06,680 --> 00:00:11,519\nhi everyone ...",no,grins uglies hides and we welcome back to anot...
254,811,emmymadeinjapan,PEZ giveaway WINNERS,stpWXStiw4k,Watch to find out if you're one of the lucky w...,"1\n00:00:08,540 --> 00:00:13,200\nhello everyb...",no,hello everybody I'm back again and it's it's s...
255,814,emmymadeinjapan,PEZ Whatcha Eating #82,yJfm6wFnr2s,[GIVEAWAY CLOSED] Eating old fashioned PEZ can...,"1\n00:00:08,470 --> 00:00:15,980\nhello everyb...",no,greetings my lovelies hi and welcome back to a...
256,815,emmymadeinjapan,Gummy Burger Set - Lunch Bag: Whatcha Eating? #81,9cbb_kdUspU,[GIVEAWAY CLOSED] A miniature gummy version of...,"1\n00:00:07,910 --> 00:00:14,639\nhi everybody...",no,everybody it's a me and I'm back and I'm still...
257,836,emmymadeinjapan,Giveaway Winners - Chocolat de Tomato,R-VvqR0EFNk,Wanna try Japanese tomato chocolate for yourse...,"1\n00:00:07,280 --> 00:00:12,870\nhello you tw...",no,frooty frooty frooty frooty frooty frooty brin...


In [3]:
data_with_captions= pd.read_pickle("emmy.pickle")

In [4]:
import random
import nltk

In [5]:
sample_size = 15
not_spon = [index for index, item in enumerate(youtube_data_from_sql.sponsored) if item == 'no']
not_spon_sample = random.sample(not_spon,sample_size)
spon = [index for index, item in enumerate(youtube_data_from_sql.sponsored) if item == 'yes']
spon_sample = random.sample(spon,sample_size)

In [6]:
train = youtube_data_from_sql.iloc[not_spon_sample + spon_sample, :]
train

,index,channel,title,video_id,description,captions,sponsored,fullCaptions
136,154,emmymadeinjapan,Emmy Eats GERMANY 11 Taste Test,fk-NHega9Lw,I'm tasting more treats from Germany including...,"1\n00:00:02,760 --> 00:00:08,460\n[Music]\n\n2...",no,driving Subway's hi and welcome back to anothe...
164,182,emmymadeinjapan,BLACK ANTS | Day 1 BUGMAS: a bug eating countd...,JmZdVeke6ck,It's Day 1 of Bugmas where I'm counting down t...,"1\n00:00:01,910 --> 00:00:09,210\n[Music]\n\n2...",no,[Music] hi everybody its Emmie and I'm back to...
117,135,emmymadeinjapan,MILKBAR B'Day Truffle Mix TEST | funfetti cake...,NDa1JrICeqI,Momofuku's Milkbar has box mixes of some of th...,"1\n00:00:00,000 --> 00:00:03,610\n[Music]\n\n2...",no,[Music] greetings my lovelies pillows and we w...
61,77,emmymadeinjapan,LAY'S Do US a Flavor 2017 Taste Test,kDFDgizU_SE,This year's Lay's Do Us a Flavor finalists inc...,"1\n00:00:00,000 --> 00:00:04,270\n[Music]\n\n2...",no,hello everybody its Emmie I am back to share a...
138,155,emmymadeinjapan,Girl Scouts CEREAL Taste Test,8e6rBhDGVvE,Imagine Thin Mint and Samoa in cereal form and...,"1\n00:00:00,330 --> 00:00:07,919\n[Music]\n\n2...",no,Canon's uglies hi it that we welcome back to a...
2,3,emmymadeinjapan,Abuelita VS. Ibarra CHAMPURRADO TASTE TEST | M...,_ffk-8m6Qao,"Containing chocolate, milk, a kiss of cinnamon...","1\n00:00:00,300 --> 00:00:07,799\n[Music]\n\n2...",no,greetings everyone it's Emmie hi I'm back to a...
25,26,emmymadeinjapan,"Thanksgiving FEAST CAKE - mashed potatoes, stu...",wNYUHuD0WAg,This is no ordinary cake. Composed of three l...,"1\n00:00:00,560 --> 00:00:05,430\n[Music]\n\n2...",no,greetings Elvis hi it's Emmy welcome back to a...
121,139,emmymadeinjapan,GLUE GUN & CHEESE | DIY Fondoodler | extrude m...,2w5W1t4gOC0,"Today we're answering the age old question, Wh...","1\n00:00:04,529 --> 00:00:09,210\ncreated my l...",no,greetings YouTube hi it's Emmy I'm back this t...
236,257,emmymadeinjapan,Flamin' Hot VS. Wild Habanero CHEETOS | VERSUS,xxBmHiHx6vA,It's spicy versus spicy in this hot Cheetos VS...,"1\n00:00:00,000 --> 00:00:04,520\n[music]\n\n2...",no,you greetings always hi it's Emmy I am back an...
167,185,emmymadeinjapan,VINTAGE KITCHEN GADGET TEST - Do They Work?,MelBWO2ZNmg,They don't make 'em like they used to. I'm te...,"1\n00:00:00,000 --> 00:00:07,930\n[Music]\n\n2...",no,[Music] greetings my lovelies hi as I mean wel...


In [7]:

stop = stopwords.words('english') + list(string.punctuation)

In [21]:
train_tokens = []
for sent in train.description:
    train_tokens.append([i for i in word_tokenize(sent.lower()) if i not in stop])

In [22]:
words = [y for x in train_tokens for y in x]

In [23]:
all_words = nltk.FreqDist(w.lower() for w in words)
word_features = list(all_words)



In [24]:
word_features

["'m",
 'tasting',
 'treats',
 'germany',
 'including',
 'christmas',
 'favorites',
 'episode',
 'emmy',
 'eats',
 'part',
 '11.',
 'big',
 'thank',
 'mr.',
 'j',
 'kindly',
 'sending',
 'specialties',
 'making',
 'possible',
 'join',
 'league',
 'adventuresome',
 'eaters',
 'find',
 'subscribe',
 'http',
 '//youtube.com/subscription_center',
 'add_user=emmymadeinjapan',
 'twitter',
 'https',
 '//twitter.com/emmymadeinjapan',
 'instagram',
 '//instagram.com/emmymade',
 'snapchat',
 'emmymade',
 'facebook',
 '//www.facebook.com/itsemmymadeinjapan/',
 'channel',
 '//bit.ly/1zk04sj',
 'playlist',
 '//bit.ly/2ksi8gr',
 'ireland',
 '2',
 'barmbrack',
 '//bit.ly/1hsgm8x',
 'ikea',
 'syrup',
 'debacle',
 '//bit.ly/1nwqacm',
 'oreo',
 'taste',
 'test',
 'fudge',
 '//bit.ly/1nei7i7',
 'scotish',
 'fruit',
 'cake',
 'ritter',
 'sport',
 'gold',
 'schatz',
 'schluckwerder',
 'edel',
 'sirup',
 'waldmeister',
 'gingerbread',
 'cookies',
 'www.konditorei-klingelhoefer.de',
 'opifera',
 'minz',
 'mi

In [25]:
filename = 'dictionary.sav'
pickle.dump(word_features, open(filename, 'wb'))

In [26]:
def document_features(document):
    document_words = document.split()
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [27]:
feature_set = []
for i in range(len(train)):
    feature_set.append((document_features(train.fullCaptions.iloc[i]), 
           train.sponsored.iloc[i]))

In [28]:
feature_set = []
for i in range(len(train)):
    feature_set.append((document_features(train.description.iloc[i]), 
           train.sponsored.iloc[i]))

In [29]:
classifier = nltk.NaiveBayesClassifier.train(feature_set)

In [31]:
classifier.show_most_informative_features(5)

Most Informative Features
           contains(hot) = True               no : yes    =      3.7 : 1.0
         contains(first) = False              no : yes    =      2.8 : 1.0
       contains(receive) = False              no : yes    =      2.4 : 1.0
           contains(3.0) = True              yes : no     =      2.3 : 1.0
          contains(take) = True               no : yes    =      2.3 : 1.0


In [13]:
train.description.iloc[['mint'in cap for cap in train.captions]]

169    On my last trip to California I stopped into C...
188    A big thank you to Progresso for sponsoring th...
101    Visit https://squarespace.com/emmy and enter t...
Name: description, dtype: object

In [55]:
spon_test = list(set(spon) - set(spon_sample))
not_spon_test =  random.sample(list(set(not_spon) - set(not_spon_sample)),len(spon_test))

test = youtube_data_from_sql.iloc[not_spon_test + spon_test, :]

test_feature_set = []
for i in range(len(test)):
    test_feature_set.append((document_features(test.fullCaptions.iloc[i]), 
           test.sponsored.iloc[i]))
    
    
nltk.classify.accuracy(classifier, test_feature_set)

0.75

In [52]:
test

,index,channel,title,video_id,description,captions,sponsored,fullCaptions
148,166,emmymadeinjapan,SPICY CHEESE RAMEN CHALLENGE,tFo-ZFxy_Tc,I found a cheesy version of the Samyang noodle...,"1\n00:00:00,000 --> 00:00:03,720\n[Music]\n\n2...",no,greetings uh plays hi it's Emmy welcome back t...
108,126,emmymadeinjapan,WAFFLE HOT DOG Recipe - Weenies,CrPXJZikzCM,Here's how to take breakfast to the next level...,"1\n00:00:00,000 --> 00:00:02,360\n[Music]\n\n2...",no,everybody it's me I'm back back in with a how ...
105,123,emmymadeinjapan,$1 CHICKEN FRIED STEAK & homemade gravy | DOLL...,RNQdkP0uUQc,Visit https://squarespace.com/emmy and enter t...,"1\n00:00:00,320 --> 00:00:07,350\n[Music]\n\n2...",yes,Greetings my lovelies~ Hi it's Emmy. Welcome b...
14,15,emmymadeinjapan,Emmy Takes the SUPERTASTER Test - counting tas...,P_8ZYxJ8ay0,Check out HELLO FRESH: http://bit.ly/2fiQBC0 a...,"1\n00:00:05,200 --> 00:00:10,200\nGreetings my...",yes,you everyone I'm back it's been a while I've n...


In [58]:
i=2
(classifier.classify(document_features(test.fullCaptions.iloc[i])), test.sponsored.iloc[i])

('yes', 'yes')

In [18]:
# save the model to disk
filename = 'sponView_classifier.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [64]:
filename = 'sponView_classifier.sav'
classifier = pickle.load(open(filename, 'rb'))
nltk.classify.accuracy(classifier, test_feature_set)

0.75

In [65]:
for i in range(4):
    print((classifier.classify(document_features(test.fullCaptions.iloc[i])), 
           test.sponsored.iloc[i],test.video_id.iloc[i]) )

('yes', 'no', 'gZe7VETOgaY')
('no', 'no', 'LL9AG9U7Vqw')
('yes', 'yes', 'RNQdkP0uUQc')
('yes', 'yes', 'P_8ZYxJ8ay0')


# flask code

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import re

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pickle

from nltk.tokenize import word_tokenize # or use some other tokenizer
from nltk.corpus import stopwords
import string

In [81]:
#### given video id pull captions
######input video_id
def get_captions(video_id):
    ###### for MVP just take from table. for next week figure out authentication and implement parsing
    dbname = 'youtubeSpon'
    username = 'april' # change this to your username

    engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
    con = None
    con = psycopg2.connect(database = dbname, user = username)

    # query:
    sql_query = """SELECT * FROM "youtubeEmmy" WHERE "youtubeEmmy".video_id = '%s';"""%video_id
    selectedCaption = pd.read_sql_query(sql_query,con)
    return selectedCaption

get_captions('P_8ZYxJ8ay0').fullCaptions

0    you everyone I'm back it's been a while I've n...
Name: fullCaptions, dtype: object

In [ ]:
def document_features(document):
    document_words = document.split()
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [49]:
#### run classifier
def run_classifier(video_id):
    #load model
    filename = 'sponView_classifier.sav'
    classifier = pickle.load(open(filename, 'rb'))
    captions = get_captions(video_id)
    return classifier.classify(document_features(captions))
    
run_classifier('QDlbOrOO0_E')

'no'

In [53]:
run_classifier('RNQdkP0uUQc')

'no'